<h1><center>Assignment 1: Training Model</center></h1><br>
<h3>Napong Leelasithorn 61070505228</h3><br>

<b> Tasks [10 points] </b>
<ol>
    <li>[1 point] Prepare the data in one-against-the-rest strategy. This can be done by converting the "Species" column into 3 binary columns</li>
    <li>[2 points] Formulate the error function of the logistic regression with ridge regularization criterion.</li>
    <li>[2 points] Derive the gradient of the error function of the logistic regression with ridge regularization criterion.</li>
    <li>[2 points] Implement the gradient descent using all of the dataset in each iteration. (Use equation from Task 3)</li>
    <li>[1 point] Implement the stochastic gredient descent using the subset of dataset in each iteration. (use equation from Task 3)</li>
    <li>[1 point] Test to see the effect of $\lambda$ on the training process.</li>
    <li>[1 point] Test to see the effect of sampling proportion in Task 5.</li> 
    

In [37]:
import pandas as pd
import numpy as np

### Task 1 [1 point]
#### Load Iris Dataset

In [38]:
iris = pd.read_csv('IRIS.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [39]:
iris = pd.concat([iris, pd.get_dummies(iris['species'])], axis = 1).drop(columns = 'species')
iris.insert(0, 'intercept', 1)
iris.head()

,intercept,sepal_length,sepal_width,petal_length,petal_width,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,1,4.9,3.0,1.4,0.2,1,0,0
2,1,4.7,3.2,1.3,0.2,1,0,0
3,1,4.6,3.1,1.5,0.2,1,0,0
4,1,5.0,3.6,1.4,0.2,1,0,0


In [40]:
iris_train = iris.sample(frac = 0.8, random_state = 1)
iris_test = iris.drop(index = iris_train.index)

In [41]:
# transform y into np.array format
# one-against-the-rest

# train
y_set_train = np.array(iris_train[['Iris-setosa']]) # setosa = 1
y_ver_train = np.array(iris_train[['Iris-versicolor']]) # versicolor = 1
y_vir_train = np.array(iris_train[['Iris-virginica']]) # virginica = 1

# test
y_set_test = np.array(iris_test[['Iris-setosa']]) # setosa = 1
y_ver_test = np.array(iris_test[['Iris-versicolor']]) # versicolor = 1
y_vir_test = np.array(iris_test[['Iris-virginica']]) # virginica = 1

y_set_test[:5]

array([[1],
       [1],
       [1],
       [1],
       [1]], dtype=uint8)

In [42]:
# transform all features into np.array format
x_train = np.array(iris_train.iloc[:,:5])
x_test = np.array(iris_test.iloc[:,:5])
# add an intercept
x_train[:5]

array([[1. , 5.8, 4. , 1.2, 0.2],
       [1. , 5.1, 2.5, 3. , 1.1],
       [1. , 6.6, 3. , 4.4, 1.4],
       [1. , 5.4, 3.9, 1.3, 0.4],
       [1. , 7.9, 3.8, 6.4, 2. ]])

### Task 2 [2 points]

<b>Logistic regression</b>
$$ H_w(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$ <br> 
Where
$$z = f(x,w) = \sum_{i = 1}^{n} w_i x_i $$

In [43]:
# implement the sigmoid function as (1)
def sigmoid(z):
    return 1/(1+np.exp(-z))

The error function for ridge regression can be expressed in the form of (cost function + L2 regularization term)<br><br>
$$ E(w) = L(w) + \lambda\sum_{i = 1}^n w_i^2 $$<br>
and $L(w)$ is,<br><br>
$$
\begin{align}
L(w) &=  \frac{1}{n}\sum_{i = 1}^{n}[{-y_i}\log{H_w}({x_i}) - (1 - {y_i})\log(1 - {H_w}({x_i}))] \\ 
&= \frac{-1}{n}\sum_{i = 1}^{n}[{y_i}\log{H_w}({x_i}) + (1 - {y_i})\log(1 - {H_w}({x_i}))] 
\end{align}
$$<br>

Therefore, <br>
$$ E(w) = \frac{-1}{n}\sum_{i = 1}^{n}[{y_i}\log{H_w}({x_i}) + (1 - {y_i})\log(1 - {H_w}({x_i}))] + \lambda\sum_{i = 1}^n w_i^2 \tag{2} $$<br>

We can calculate this error function using matrix form instead of processing a single row, like this <br><br><br>
$$E(w) = \frac{-1}{n} \times \left(\mathbf{Y}^T \cdot log(\mathbf{H_w(X\cdot{W})}) + \mathbf{(1-Y)}^T \cdot log(\mathbf{1-H_w(X\cdot{W})}) \right) + \lambda (\mathbf{W}^T\mathbf{W}) $$<br>

In [81]:
# Formulate the error function.
def errorfunction(x, y, w, _lambda):
    n = len(x)
    h = sigmoid(x @ w)
    return (-1/n)*((y.transpose() @ np.log(h)) + (1-y).transpose() @ np.log(1-h)) \
            + (_lambda*(w.transpose() @ w)), h  # return the error function and Sigmoid of (xw)

<br><br>
$$ \frac{-1}{n}  \times
\left (
\begin{bmatrix} 
y_1 & y_2 & {..} & y_n\\
\end{bmatrix}
\begin{bmatrix} 
log(H_w(f(x_1,w_1)))\\
log(H_w(f(x_2,w_2)))\\
...\\
log(H_w(f(x_n,w_n)))
\end{bmatrix}
\right )
\times
\begin{bmatrix} 
{1 - y_1} & 1-y_2 & {..} & 1-y_n
\end{bmatrix}
\begin{bmatrix} 
1-log(H_w(f(x_1,w_1)))\\
1-log(H_w(f(x_2,w_2)))\\
...\\
1-log(H_w(f(x_n,w_n)))
\end{bmatrix} + \lambda
\begin{bmatrix} 
w_1 & w_2 & {..} & w_n
\end{bmatrix}
\begin{bmatrix} 
w_1 \\ w_2 \\ {..} \\ w_n
\end{bmatrix}
$$ <br><br>

### Task 3 [2 points]

I will use the <b>equation (2)</b> to find the partial derivative of the error function in <b>Task 2</b> <br>
Let's derive the gradient of the $E(w)$ function.

$$ E(w) = \frac{-1}{n}\sum_{i = 1}^{n}[{y_i}\log{H_w}({x_i}) + (1 - {y_i})\log(1 - {H_w}({x_i}))] + \lambda\sum_{i = 1}^n w_i^2 $$

Then<br><br>
$$
\begin{align}
\frac{\partial E(w)}{\partial w} &= \frac{\partial}{\partial w}(\frac{-1}{n}\sum_{i = 1}^{n}[{y_i}\log{H_w}({x_i}) + (1 - {y_i})\log(1 - {H_w}({x_i}))] + \lambda\sum_{i = 1}^n w_i^2) \\
&= \frac{-1}{n}\left(\frac{\partial}{\partial w}\sum_{i = 1}^{n}[{y_i}\log{H_w}({x_i}) + (1 - {y_i})\log(1 - {H_w}({x_i}))] \right) + \lambda\frac{\partial}{\partial w}\sum_{i = 1}^n w_i^2\tag{3}
\end{align}
$$<br>

I will apply the partial derivertive into these 2 terms in (3) separately.<br>
For the first term, we should calculate the derivative of sigmoid function (1) before take a look at the derivative of cost function.

Sigmoid function
$$
H(z) =  \frac{1}{1+\exp^{-z}}
$$

diff,
$$ H(z)' = \left( \frac{1}{1+\exp^{-z}} \right)' = \frac{-(1+e^{-z})'}{(1+e^{-z})^2} = \frac{0- (e^{-z})'}{(1+e^{-z})^2} = \frac{-(-z)'(e^{-z})}{(1+e^{-z})^2} = \frac{1(e^{-z})}{(1+e^{-z})^2} $$

Note that in this step, if it is partial derivative with respect to "w", because of the chain rule there will be an additional multiplication term which is $x$ from the partial derivertive of $f(x,w)$ respect to $w$

Continue
$$
\begin{align}
H(z)' &= \frac{e^{-z}}{(1 + e^{-z})^2} \\\\
& = \left( \frac{1}{1 + e^{-z}} \right ) \left ( \frac{e^{-z}}{1 + e^{-z}} \right ) \\\\
& = H(z) \left ( \frac{+1-1+e^{-z}}{1+e^{-z}} \right ) \\\\
& = H(z) \left ( \frac{1+e^{-z}}{1+e^{-z}} - \frac{1}{1+e^{-z}} \right ) \\\\
& = H(z)(1 - H(z)) 
\end{align}
$$

Multiply the above equation with $x_i$ term for the partial derivative with respect to $w$ <br>
Therefore, we will get this <br><br>
$$ \frac{\partial}{\partial w}H_w(x_i) = H_w(x_i)(1-H_w(x_i))x_i \tag{4}
$$

Now, I will try to apply the partial derivative with the cost/loss function.

$$
\begin{align}
\frac{\partial L(w)}{\partial w} &= \frac{-1}{n}\left(\frac{\partial}{\partial w}\sum_{i = 1}^{n}[{y_i}\log{H_w}({x_i}) + (1 - {y_i})\log(1 - {H_w}({x_i}))]\right) \\\\
&= \frac{-1}{n}\sum_{i = 1}^{n}[{y_i}\frac{\partial}{\partial w}\log{H_w}({x_i})+ (1 - {y_i})\frac{\partial}{\partial w}\log(1 - {H_w}({x_i}))] \\\\
&= \frac{-1}{n}\sum_{i = 1}^{n} \left[ \frac{{y_i}\frac{\partial}{\partial w}{H_w}({x_i})}{{H_w}({x_i})} + \frac{(1 - {y_i})\frac{\partial}{\partial w}(1 - {H_w}({x_i}))}{(1 - {H_w}({x_i}))} \right ]
\end{align}
$$
<br>

Substitute (4), $\frac{\partial}{\partial w}H_w(x_i) = H_w(x_i)(1-H_w(x_i))x_i$ in this step,<br>
Then we will get<br><br><br>
$$
\begin{align}
\frac{\partial L(w)}{\partial w} &= \frac{-1}{n}\sum_{i = 1}^{n} \left[ \frac{{y_i}H_w(x_i)(1-H_w(x_i))x_i}{{H_w}({x_i})} + \frac{-(1 - {y_i})H_w(x_i)(1-H_w(x_i))x_i}{(1 - {H_w}({x_i}))} \right ]\\\\
&= \frac{-1}{n}\sum_{i = 1}^{n}[{y_i}(1-H_w(x_i)) -(1 - {y_i})H_w(x_i)]x_i \\\\
&= \frac{-1}{n}\sum_{i = 1}^{n}[y_i - y_iH_w(x_i) - H_w(x_i) + y_iH_w(x_i)]x_i \\\\
&= \frac{-1}{n}\sum_{i = 1}^{n}(y_i - H_w(x_i))x_i \\\\
&= \frac{1}{n}\sum_{i = 1}^{n}(H_w(x_i)-y_i)x_i \tag{5}
\end{align}
$$

and the L2 regularization term.<br><br>
$$\lambda\frac{\partial}{\partial w}\sum_{i = 1}^n w_i^2 = 2\lambda\sum_{i=1}^{n}w_i\tag{6} $$

Combine these two terms (5)(6) to get the partial derivative of $E(w)$,<br><br><br>
$$\frac{\partial E(w)}{\partial w} =\frac{1}{n}\sum_{i = 1}^{n}(H_w(x_i)-y_i)x_i + 2\lambda\sum_{i=1}^{n}w_i \tag{*}$$<br><br><br>

### Task 4
Implement gradient descent using equation in task 3.

It can be implement by this vectorized version:<br><br>
$$ \triangledown E(w) = \frac{1}{n} \times X^T \cdot({H(X\cdot W)} - Y) + 2\lambda W$$

So, we'll update "w" as this equation<br><br>
$$\mathbf{w} = \mathbf{w} - \frac{\alpha}{n} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right) + 2 \lambda \mathbf{w}$$

In [82]:
def gradientdescent(x, y, w, _lambda, iterations, alpha):
    n = len(x)
    for i in range(0, iterations):
        loss,h = errorfunction(x,y,w,_lambda)
        w = w - alpha*(x.transpose() @ (h-y)) + 2*_lambda*w
        print("Error:{}".format(float(loss)))
    return w

<br><br>
### Task 5

Implement stochastic gradient descent.<br>
From Task 7 want me to test the sampling proportion from this task, <br>
So, I decide to implement mini-batch gradient descent instead.

In [83]:
def minibatchGD(x, y, w, _lambda, iterations, alpha, numsampling):
    n = len(x)
    for i in range(0, iterations):
        
        # create an array which has length = number of row x
        index = np.arange(x.shape[0])
        
        # shuffle index
        np.random.shuffle(index)
        
        # match the index which already shuffled with x and y, and the number of samples = numsampling
        x_ran, y_ran = x[index[:numsampling+1]], y[index[:numsampling+1]]
        
        loss,h = errorfunction(x_ran,y_ran,w,_lambda)
        w = w - alpha*(x_ran.transpose() @ (h-y_ran)) + 2*_lambda*w
        print("Error:{}".format(float(loss)))
    return w

<br><br><br>
### Task 6
Test to see the effect of $\lambda$ on training process

Initialize w with the all-ones matrix

In [102]:
w = np.ones((x_train.shape[1],1))
w

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [85]:
# Accuracy test for gradient descent gradient descent
from sklearn.metrics import classification_report

def accuracy_test(x, y, w):
    h = sigmoid(x@w)
    h = np.where(h > 0.5, 1., h)
    h = np.where(h < 0.5, 0., h)
    return classification_report(y, h)

As we know that, $\lambda$ term can prevent overfitting.<br>
So, I build 2 different models with lambda = 0 and 0.05 and other all parameters are the same.(Using normal gradient descent.)

#### Train accuracy for the 1st model

In [111]:
print(accuracy_test(x_train, y_set_train,gradientdescent(x_train, y_set_train, w,  _lambda = 0, iterations =20, alpha = 0.0005)))

Error:11.27417257251455
Error:9.221586689153398
Error:7.1693047370714496
Error:5.1194608705332145
Error:3.0917231317009577
Error:1.2840795940860943
Error:0.5592519662689854
Error:0.5180139113586185
Error:0.49703386797246185
Error:0.4780170994557533
Error:0.46011091782392566
Error:0.4431976727772849
Error:0.4272177617264853
Error:0.41211764298709735
Error:0.3978457121489225
Error:0.38435227561672947
Error:0.3715897735541381
Error:0.35951293557534003
Error:0.348078864696324
Error:0.3372470651815217
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        80
           1       1.00      0.85      0.92        40

   micro avg       0.95      0.95      0.95       120
   macro avg       0.97      0.93      0.94       120
weighted avg       0.95      0.95      0.95       120



#### Test accuracy for the 1st model

In [112]:
print(accuracy_test(x_test, y_set_test,gradientdescent(x_train, y_set_train, w,  _lambda = 0, iterations =20, alpha = 0.0005)))

Error:11.27417257251455
Error:9.221586689153398
Error:7.1693047370714496
Error:5.1194608705332145
Error:3.0917231317009577
Error:1.2840795940860943
Error:0.5592519662689854
Error:0.5180139113586185
Error:0.49703386797246185
Error:0.4780170994557533
Error:0.46011091782392566
Error:0.4431976727772849
Error:0.4272177617264853
Error:0.41211764298709735
Error:0.3978457121489225
Error:0.38435227561672947
Error:0.3715897735541381
Error:0.35951293557534003
Error:0.348078864696324
Error:0.3372470651815217
              precision    recall  f1-score   support

           0       0.83      1.00      0.91        20
           1       1.00      0.60      0.75        10

   micro avg       0.87      0.87      0.87        30
   macro avg       0.92      0.80      0.83        30
weighted avg       0.89      0.87      0.86        30



At this point, you can see a little bit overfitting without lambda (=0).<br>

#### Train accuracy for the 2nd model

In [115]:
# 1st model using _lambda = 0 to predict trainset
print(accuracy_test(x_train, y_set_train,gradientdescent(x_train, y_set_train, w,  _lambda = 0.05, iterations =20, alpha = 0.0005)))

Error:11.52417257251455
Error:10.583224299643394
Error:9.552585390320555
Error:8.424267594033083
Error:7.189842037562974
Error:5.841012017370671
Error:4.372988399895765
Error:2.8092069194794886
Error:1.3984480298265627
Error:0.9091424056762596
Error:0.9340788207720097
Error:1.0017104149739018
Error:1.0899242232770134
Error:1.201067925266464
Error:1.3401856055549985
Error:1.5134973252765846
Error:1.7284276096387263
Error:1.9938591006921964
Error:2.320457263590746
Error:2.7210570825593807
              precision    recall  f1-score   support

           0       0.85      0.95      0.90        80
           1       0.87      0.68      0.76        40

   micro avg       0.86      0.86      0.86       120
   macro avg       0.86      0.81      0.83       120
weighted avg       0.86      0.86      0.85       120



#### Test accuracy for the 2nd model

In [116]:
# 2nd model using _lambda = 0 to predict testset
print(accuracy_test(x_test, y_set_test,gradientdescent(x_train, y_set_train, w,  _lambda = 0.05, iterations =20, alpha = 0.0005)))

Error:11.52417257251455
Error:10.583224299643394
Error:9.552585390320555
Error:8.424267594033083
Error:7.189842037562974
Error:5.841012017370671
Error:4.372988399895765
Error:2.8092069194794886
Error:1.3984480298265627
Error:0.9091424056762596
Error:0.9340788207720097
Error:1.0017104149739018
Error:1.0899242232770134
Error:1.201067925266464
Error:1.3401856055549985
Error:1.5134973252765846
Error:1.7284276096387263
Error:1.9938591006921964
Error:2.320457263590746
Error:2.7210570825593807
              precision    recall  f1-score   support

           0       0.83      1.00      0.91        20
           1       1.00      0.60      0.75        10

   micro avg       0.87      0.87      0.87        30
   macro avg       0.92      0.80      0.83        30
weighted avg       0.89      0.87      0.86        30



Here, we can see the difference between train accuracy and test accuracy in both model is dicreasing 

<br>Therefore, adding regularization term can preventing overfit or we can just saying that it can make the model to not too fit with the data by adding some noise <br><br>

### Task 7

Test the effect of sampling proportion of mini-batch gradient descent.<br><br>
<b>Start with number of samples = 30</b>

In [157]:
# numsampling = 30
print(accuracy_test(x_test, y_set_test,minibatchGD(x_train, y_set_train, w, 
                                                   _lambda = 0.003, 
                                                   iterations =20,
                                                   alpha = 0.003,
                                                   numsampling = 100)))

Error:11.423916390344004
Error:1.1122206082662156
Error:1.952740526136283
Error:0.5492276305084025
Error:0.4678279478950846
Error:0.7277792122925285
Error:0.21070520921059657
Error:0.2076671628070851
Error:0.1743499285147525
Error:0.17176086809004715
Error:0.1491575193852042
Error:0.1314835613724362
Error:0.11934516814023644
Error:0.11501096664576202
Error:0.11004996652339073
Error:0.10266058116415856
Error:0.09950076643203011
Error:0.09209631333863712
Error:0.09054300055382575
Error:0.08608228540599312
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        10

   micro avg       1.00      1.00      1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



As you can see, this model seem to stucked at local minimum as it always decreasing over iterations, but I think that this dataset's too pretty. So, we cannot easily make it obvious overfitting at testset (always 1.0 for train and test accuracy).

<br>
But if we look at this following stochastic gradient descent<br>
<b> number of sample = 1 </b><br>
at error value in line 2 (0.0144), it about to stucks at local minimum but the random sample help to prevent SGD going down and escape that local minimum.

In [149]:
# numsampling = 1
print(accuracy_test(x_train, y_set_train,minibatchGD(x_train, y_set_train, w, 
                                                   _lambda = 0.003,
                                                   iterations =20,
                                                   alpha = 0.003,
                                                   numsampling = 1)))

Error:14.415000568684032
Error:9.928000779024883
Error:0.01449706646619383
Error:16.52015647180086
Error:7.990640497609038
Error:16.255483080921714
Error:15.970138324689685
Error:7.578490758678858
Error:14.759352904229212
Error:16.095260322098333
Error:13.36557125179011
Error:5.927972814525737
Error:6.52009533561195
Error:14.323090761699204
Error:14.07560906192835
Error:6.391011205233222
Error:14.043714665615875
Error:11.274487485400167
Error:6.174755425285734
Error:12.81443938681103
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        80
           1       0.33      1.00      0.50        40

   micro avg       0.33      0.33      0.33       120
   macro avg       0.17      0.50      0.25       120
weighted avg       0.11      0.33      0.17       120



This model has only 33% accuracy because I use only 20 iterations in this model. If you want to find out the better one from SGD you can change another parameters like alpha, iterations to find the actual global minimum<br>
#### like this

In [155]:
print(accuracy_test(x_test, y_set_test,minibatchGD(x_train, y_set_train, w, 
                                                   _lambda = 0.003, 
                                                   iterations =100,
                                                   alpha = 0.005,
                                                   numsampling = 1)))

Error:17.565000208303907
Error:16.899195998737934
Error:0.013340043214273086
Error:8.895112928063805
Error:14.300751981450292
Error:0.012482415492046857
Error:16.12267658240705
Error:13.39106211713242
Error:10.987255778678206
Error:12.473460304137573
Error:10.823815630265756
Error:11.62019665713977
Error:5.661552036471206
Error:5.591279578126396
Error:5.410158855052009
Error:0.009136214124102308
Error:4.328289859150251
Error:4.305660729842606
Error:9.669978367734753
Error:0.012509749148180498
Error:3.81797652902965
Error:7.422764337750684
Error:7.920374610097718
Error:5.85722931077063
Error:6.957609145647011
Error:6.05845362194369
Error:3.8866685808602064
Error:1.8078535861396183
Error:3.500307251863104
Error:2.849932274465764
Error:2.3766924162991745
Error:0.2877638021372176
Error:2.0459861247655295
Error:1.4483062338991473
Error:1.0284250324151092
Error:0.5718413479952249
Error:0.6282722418806874
Error:0.3454219798969302
Error:0.49265360650315826
Error:0.6953426800183896
Error:0.3339